In [ ]:
!pip install pillow
!pip install pytesseract
!pip install opencv-python-headless
!pip install imutils

In [ ]:
# in a terminal: add-apt-repository ppa:alex-p/tesseract-ocr
!apt install -y tesseract-ocr-grc

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import imutils
import os
import shutil
import pytesseract
import re
import json

def ocrRoster(filename):
    filepath = 'matches/roster/' + filename
    im_rgb = cv.imread(filepath)
    im_gray = cv.cvtColor(im_rgb, cv.COLOR_BGR2GRAY)
    im_thresh = cv.threshold(im_gray,155,255,cv.THRESH_BINARY_INV)[1]

    lower = np.array([0])
    upper = np.array([15])
    shapeMask = cv.inRange(im_thresh, lower, upper)

    cnts = cv.findContours(shapeMask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    for c in cnts:
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.04 * peri, True)

        if len(approx) == 4:
            (x, y, w, h) = cv.boundingRect(approx)
            ar = w / float(h)

            if ar >= .1 and ar <= .3 and h > 25:
                crop = im_thresh[approx[2][0][1] - 40 : approx[2][0][1] , approx[0][0][0] - 415 : approx[0][0][0] - 10]
                try:
                    text = pytesseract.image_to_string(crop)
                    first = text.split('\n')[0]
                    ascii = first.encode('ascii', 'ignore').strip().split(' ')
                    sans_special = ''
                    for string in ascii:
                        if bool(re.match('[a-zA-Z0-9]+$', string)):
                            sans_special += string + ' '
                    stripped = sans_special.strip()
                    q_check = stripped.split(' ')
                    # Sometimes, the audio icon reads as a 'q', so...
                    if q_check[-1] == 'q':
                        q_check[-1] = ''
                        stripped = ' '.join(q_check).strip()
                    twitch_id = filename.split('.')[0]
                    if stripped:
                        if os.path.isdir('d2-stream-name-parser/' + stripped) == False:
                            os.mkdir('d2-stream-name-parser/' + stripped)
                        data = {}
                        try:
                            with open('d2-stream-name-parser/' + stripped + '/twitch.json') as f:
                                for i in f:
                                    data = json.loads(i)

                        except:
                            print('new file!')
                        if twitch_id in data:
                            data[twitch_id] += 1
                        else:
                            data[twitch_id] = 1
                        with open('d2-stream-name-parser/' + stripped + '/twitch.json', 'w+') as f:
                            f.write(json.dumps(data))
                except:
                    print('issue parsing text')
                break
    try:
        os.remove(filepath)
    except:
        print('error removing files')

for filename in os.listdir('matches/roster'):
    if not filename.startswith('.'):
        ocrRoster(filename)